In [ ]:
# import torch
# import cv2
# from segment_anything import sam_model_registry
# from segment_anything import SamPredictor

# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MODEL_TYPE = "vit_h"

# sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)
# sam.to(device=DEVICE)

# mask_predictor = SamPredictor(sam)

# image_bgr = cv2.imread(IMAGE_PATH)
# image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
# mask_predictor.set_image(image_rgb)

# box = np.array([70, 247, 626, 926])
# masks, scores, logits = mask_predictor.predict(box=box, multimask_output=True)